In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
url_donna = "https://www.kammi.it/scarpe-donna/"
url_d_sneakers = "stivali-stivaletti/" #se non uso questo pezzo di url posso 
url = url_donna + url_d_sneakers
print(url)

https://www.kammi.it/scarpe-donna/stivali-stivaletti/


In [3]:
# Scrolla la pagina lentamente fino in fondo
def scroll_to_bottom(driver, pause_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scrolla in fondo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)  # Aspetta che la pagina carichi nuovi contenuti
        # Calcola la nuova altezza e confrontala
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # è il "pilota" che fa le azioni sul browser  
# driver.maximize_window() # apre e massimizza Chrome
driver.get(url) # apre la pagina su Chrome

In [5]:
# Scrolla fino in fondo per caricare tutti i prodotti
scroll_to_bottom(driver)

In [6]:
soup = BeautifulSoup(driver.page_source)

In [7]:
# print(soup)

In [8]:
# cerco i nomi dei prodotti
names_dirty = soup.find_all("p", class_="product-title")
# print(nomi_dirty)

In [9]:
len(names_dirty)

2

In [10]:
print(names_dirty)

[<p class="product-title">Tronchetto 6022</p>, <p class="product-title">Tronchetto 6022</p>]


In [11]:
nomi_prodotti_home = []
for nome in names_dirty:
    nome = nome.text.strip()
    nomi_prodotti_home.append(nome)
print(nomi_prodotti_home)

['Tronchetto 6022', 'Tronchetto 6022']


In [12]:
len(nomi_prodotti_home)

2

In [13]:
# cerco i link dei prodotti
div_link = soup.find_all ("div", class_="col-6 col-md-4 col-lg-3 col-xl-auto mt-4")
print(len(div_link))

2


In [14]:
div_link[0].find("a")

<a href="https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21357.html" onclick="return productClick('elenco_prodotti_tronchetto-6022_00005334001', 0, this);">
<!-- VARIABILI PER GTM -->
<input class="codiceProdottoGTM" type="hidden" value="tronchetto-6022_00005334001"/>
<input class="titoloGTM" type="hidden" value="Tronchetto 6022"/>
<input class="prezzoGTM" type="hidden" value="125.10"/>
<input class="categoriaGTM" type="hidden" value="Stivali e stivaletti"/>
<input class="brandGTM" type="hidden" value="Kammi"/>
<input class="listaGTM" type="hidden" value="Scarpe da donna estive - Stivali e stivaletti"/>
<img class="mditaly" src="https://www.kammi.it/img/madeInItaly_vert.jpg"/>
<div class="product-image d-flex align-items-center justify-content-center">
<img alt="Tronchetto 6022" class="img-fluid" src="https://www.kammi.it/ImmaginiProdottiP/tronchetto-6022_21385_277x277.jpg"/>
<img alt="Tronchetto 6022" class="img-fluid img-2" src="https://www.kammi.it

In [15]:
a_link = []
for i in div_link:
    a = i.find("a")
    a_link.append(a)
print(len(a_link))
# a_link[0]

2


In [16]:
a_link[0].get("href")

'https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21357.html'

In [17]:
link_prodotti =[]
for i in a_link:
    link = i.get("href")
    link_prodotti.append(link)
print(len(link_prodotti))
print(link_prodotti)

2
['https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21357.html', 'https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21356.html']


In [18]:
url_prodotto = link_prodotti[0]
print(url_prodotto)

https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21357.html


In [19]:
req = requests.get(url_prodotto)

In [20]:
soup_p = BeautifulSoup(req.text)

In [21]:
# cerco diverse cose del prodotto
#soup_p.find_all("h1", class_="text-black h4 fw-bold")
# soup_p.find_all("p", class_="mt-4")
# soup_p.find_all("p", class_="price mt-2")
soup_p.find_all("button", class_="content d-flex align-items-center justify-content-center p-2 checkRitiroPP")
#soup_p.find_all("div", class_="option-color")

[<button class="content d-flex align-items-center justify-content-center p-2 checkRitiroPP" type="button">35</button>,
 <button class="content d-flex align-items-center justify-content-center p-2 checkRitiroPP" type="button">37</button>,
 <button class="content d-flex align-items-center justify-content-center p-2 checkRitiroPP" type="button">38</button>,
 <button class="content d-flex align-items-center justify-content-center p-2 checkRitiroPP" type="button">39</button>,
 <button class="content d-flex align-items-center justify-content-center p-2 checkRitiroPP" type="button">40</button>]

In [22]:
contatore = 1 
totale = len(link_prodotti)
nomi = []
descrizioni = []
prezzi_interi = []
taglie = []
colori = []
rows = []  # List to store all rows for each color/size combination

for link in link_prodotti:
    print(link, str(contatore) + " / " + str(totale))
    contatore += 1
    req = requests.get(link)
    soup_p = BeautifulSoup(req.text, 'html.parser')

    # Get the product name
    nome = soup_p.find("h1", class_="text-black h4 fw-bold")
    nome = nome.text.strip()
    print(nome)
    nomi.append(nome)

    # Get and clean the description
    descrizione = soup_p.find("p", class_="mt-4")
    descrizione = descrizione.text
    descrizione = descrizione.replace("\n", "")
    start = descrizione.find("in")  # Find "in"
    end = descrizione.find(".") if descrizione.find(".") != -1 else len(descrizione)
    if start != -1 and end != -1:
        descrizione = descrizione[start:end].strip()
    descrizione = descrizione.replace("in pelle", "").replace("in nappa", "").strip()
    descrizioni.append(descrizione)

    # Get and clean the price
    intero = soup_p.find("p", class_="price mt-2")
    intero = intero.text.replace("€", "").replace(",", ".").strip()
    prezzi_interi.append(intero)

    # Extract colors and sizes
    colori_p = soup_p.find_all("div", class_="option-color")
    colori_temp = []
    for colore in colori_p:
        # Check if the color is selected
        if "selected" in colore.get("class", []):
            # Extract selected color (background color)
            div = colore.find("div", attrs={"style": True})
            if div and "style" in div.attrs:
                style = div["style"]
                start = style.find("background:#") + len("background:#")
                end = style.find(";", start)
                if start != -1 and end != -1:
                    codice = style[start:end].strip()
                    colori_temp.append(codice)  # Add the selected color
        else:
            # Extract other colors (for non-selected)
            div = colore.find("div", attrs={"style": True})
            if div and "style" in div.attrs:
                style = div["style"]
                start = style.find("background:#") + len("background:#")
                end = style.find(";", start)
                if start != -1 and end != -1:
                    codice = style[start:end].strip()
                    colori_temp.append(codice)  # Add other color
    colori.append(colori_temp)  # Append colors for this product
    print(colori_temp)

    # Extract available sizes for each color
    taglie_p = soup_p.find_all("button", class_="content d-flex align-items-center justify-content-center p-2 checkRitiroPP")
    taglie_temp = []
    for i in taglie_p:
        t = i.text.strip()
        taglie_temp.append(t)
    taglie.append(taglie_temp)
    print(taglie_temp)

    # Save color/size combinations in rows
    for color in colori_temp:
        for size in taglie_temp:
            rows.append({
                'name': nome,                  # Product name
                'color': color,                 # Color option
                'size': size,               # Size option (taking the first 2 characters if needed)
                'price': intero             # Original price (remove "€" sign)
            })

print(len(nomi))
print(nomi)
print(len(descrizioni))
print(descrizioni)
print(len(prezzi_interi))
print(prezzi_interi)
print(len(taglie))
print(taglie)
print(len(colori))
print(colori)

https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21357.html 1 / 2
Tronchetto 6022
['FFFFFF', 'f5d7b3']
['35', '37', '38', '39', '40']
https://www.kammi.it/scarpe-donna/stivali-stivaletti/tronchetto/tronchetto-6022-idp21356.html 2 / 2
Tronchetto 6022
['FFFFFF', 'f5d7b3']
['35', '37', '40']
2
['Tronchetto 6022', 'Tronchetto 6022']
2
['in morbida pelle colore nude, tacco 7 cm e zip posteriore', 'in morbida pelle colore latte, tacco 7 cm e zip posteriore']
2
['125.10', '125.10']
2
[['35', '37', '38', '39', '40'], ['35', '37', '40']]
2
[['FFFFFF', 'f5d7b3'], ['FFFFFF', 'f5d7b3']]


In [23]:
kammi_estive_donne_stivali = pd.DataFrame(rows)

In [24]:
duplicates_subset = kammi_estive_donne_stivali.duplicated(subset=['name', 'color', 'size'])

# Count how many duplicates there are based on these columns
duplicate_subset_count = duplicates_subset.sum()

print(f"Number of duplicate rows based on 'name', 'color', 'size': {duplicate_subset_count}")

Number of duplicate rows based on 'name', 'color', 'size': 6


In [25]:
duplicates_subset = kammi_estive_donne_stivali[kammi_estive_donne_stivali.duplicated(subset=['name', 'color', 'size'])]
print("Duplicate rows based on 'name', 'color', and 'size':")
print(duplicates_subset)

Duplicate rows based on 'name', 'color', and 'size':
               name   color size   price
10  Tronchetto 6022  FFFFFF   35  125.10
11  Tronchetto 6022  FFFFFF   37  125.10
12  Tronchetto 6022  FFFFFF   40  125.10
13  Tronchetto 6022  f5d7b3   35  125.10
14  Tronchetto 6022  f5d7b3   37  125.10
15  Tronchetto 6022  f5d7b3   40  125.10


In [26]:
kammi_estive_donne_stivali = kammi_estive_donne_stivali.drop_duplicates(subset=['name', 'color', 'size'])

# You can print the result to check
print(kammi_estive_donne_stivali)

              name   color size   price
0  Tronchetto 6022  FFFFFF   35  125.10
1  Tronchetto 6022  FFFFFF   37  125.10
2  Tronchetto 6022  FFFFFF   38  125.10
3  Tronchetto 6022  FFFFFF   39  125.10
4  Tronchetto 6022  FFFFFF   40  125.10
5  Tronchetto 6022  f5d7b3   35  125.10
6  Tronchetto 6022  f5d7b3   37  125.10
7  Tronchetto 6022  f5d7b3   38  125.10
8  Tronchetto 6022  f5d7b3   39  125.10
9  Tronchetto 6022  f5d7b3   40  125.10


In [28]:
kammi_estive_donne_stivali.to_csv('kammi_estive_donne_stivali.csv',index=False)